## Importing libraries

In [323]:
import random
import pandas as pd
import random
import os
import json
import re
from IPython import get_ipython

## Extracting Strategies

In [324]:
# Function to extract the single function from a given IPYNB file and get its name
def extract_function_from_ipynb(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        notebook_content = json.load(f)
    
    function_code = ""
    function_name = None
    function_name_pattern = re.compile(r"def\s+(\w+)\s*\(.*\):")
    
    for cell in notebook_content['cells']:
        if cell['cell_type'] == 'code':
            code = ''.join(cell['source'])
            function_code += code + '\n'
            match = function_name_pattern.search(code)
            if match:
                function_name = match.group(1)
    
    return function_code, function_name

# Function to run the extracted functions
def run_extracted_functions(function_code):
    ipython = get_ipython()
    ipython.run_cell(function_code)

# Path to the folder containing IPYNB files
folder_path = 'Anonymized_Strategies'

all_function_code = ""
strategies_Name = []

# Iterate through all IPYNB files in the specified folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.ipynb'):
        file_path = os.path.join(folder_path, file_name)
        function_code, function_name = extract_function_from_ipynb(file_path)
        all_function_code += function_code + '\n'
        if function_name:
            strategies_Name.append(function_name)

# Run all the extracted function code
run_extracted_functions(all_function_code)

# Print the list of function names
print("Extracted function names (strategies):", strategies_Name)

Extracted function names (strategies): ['Student_001', 'Student_002', 'Student_003', 'Student_004', 'Student_005', 'Student_006', 'Student_007', 'Student_008', 'Student_009', 'Student_010', 'Student_011', 'Student_012', 'Student_013', 'Student_014', 'Student_015', 'Student_016', 'Student_017', 'Student_018', 'Student_019', 'Student_020', 'Student_021', 'Student_022', 'Student_023', 'Student_024']


In [325]:
# changing the strings to function names
strategies = [globals()[name] for name in strategies_Name]
strategies

[<function __main__.Student_001(history)>,
 <function __main__.Student_002(history)>,
 <function __main__.Student_003(history)>,
 <function __main__.Student_004(history)>,
 <function __main__.Student_005(history)>,
 <function __main__.Student_006(history)>,
 <function __main__.Student_007(history)>,
 <function __main__.Student_008(history)>,
 <function __main__.Student_009(history)>,
 <function __main__.Student_010(history)>,
 <function __main__.Student_011(history)>,
 <function __main__.Student_012(history)>,
 <function __main__.Student_013(history)>,
 <function __main__.Student_014(history)>,
 <function __main__.Student_015(history)>,
 <function __main__.Student_016(history)>,
 <function __main__.Student_017(history)>,
 <function __main__.Student_018(history)>,
 <function __main__.Student_019(history)>,
 <function __main__.Student_020(history)>,
 <function __main__.Student_021(history)>,
 <function __main__.Student_022(history)>,
 <function __main__.Student_023(history)>,
 <function 

In [326]:
print("Number of players(Strategies):", len(strategies))

Number of players(Strategies): 24


An altered version of play function for gathering information in competition

In [191]:
def play_game(strategy1, strategy2):

    score1 = 0
    score2 = 0

    for _ in range(20):
        move1 = strategy1(history)
        flipped_history = [(move2, move1) for move1, move2 in history]
        move2 = strategy2(flipped_history)

        if move1 == "Cooperate" and move2 == "Cooperate":
            score1 += 3
            score2 += 3
        elif move1 == "Cooperate" and move2 == "Defect":
            score1 += 0
            score2 += 5
        elif move1 == "Defect" and move2 == "Cooperate":
            score1 += 5
            score2 += 0
        elif move1 == "Defect" and move2 == "Defect":
            score1 += 1
            score2 += 1

        history.append((move1, move2))

    if score1 > score2:
        return strategy1.__name__, strategy2.__name__, score1, score2, strategy1.__name__ + " wins"
    elif score2 > score1:
        return strategy1.__name__, strategy2.__name__, score1, score2, strategy2.__name__ + " wins"
    else:
        return strategy1.__name__, strategy2.__name__, score1, score2, "It's a tie"

# Main competition

Since some strategies are not deterministic, we use a specified random seed to generate numbers, which ensures the results are reproducible.

In [320]:
# Generate a random integer between 0 and 50
random_seed = random.randint(0, 100)
random_seed

90

The random number is 90.

In [321]:
# Using the predefined random seed for reproducibility
random.seed(random_seed)

# Initialize results storage
results = []
histories = []

# Play games between each unique pair of strategies
for i in range(len(strategies)):
    for j in range(i + 1, len(strategies)):
        strategy1 = strategies[i]
        strategy2 = strategies[j]
        history = []
        result = play_game(strategy1, strategy2)
        results.append(result)
        histories.append(history)

# Create DataFrame
df = pd.DataFrame(results, columns=["Strategy 1", "Strategy 2", "Score 1", "Score 2", "Result"])

# Calculate score summations, wins, losses, draws
summary = []
for strategy in strategies:
    name = strategy.__name__
    total_score = df.loc[df["Strategy 1"] == name, "Score 1"].sum() + df.loc[df["Strategy 2"] == name, "Score 2"].sum()
    wins = len(df[(df["Strategy 1"] == name) & (df["Result"] == name + " wins")]) + len(df[(df["Strategy 2"] == name) & (df["Result"] == name + " wins")])
    losses = len(df[(df["Strategy 1"] == name) & (df["Result"] != name + " wins") & (df["Result"] != "It's a tie")]) + len(df[(df["Strategy 2"] == name) & (df["Result"] != name + " wins") & (df["Result"] != "It's a tie")])
    draws = len(df[(df["Strategy 1"] == name) & (df["Result"] == "It's a tie")]) + len(df[(df["Strategy 2"] == name) & (df["Result"] == "It's a tie")])
    
    summary.append((name, total_score, wins, losses, draws))

summary_df = pd.DataFrame(summary, columns=["Strategy", "Total Score", "Wins", "Losses", "Draws"])

summary_df = summary_df.sort_values(by='Total Score', ascending=False)
summary_df

,Strategy,Total Score,Wins,Losses,Draws
4,Student_005,1194,0,7,16
2,Student_003,1188,0,9,14
7,Student_008,1186,0,6,17
23,Student_024,1185,0,8,15
8,Student_009,1185,0,9,14
22,Student_023,1177,0,6,17
3,Student_004,1175,0,7,16
12,Student_013,1172,0,8,15
14,Student_015,1170,0,6,17
1,Student_002,1165,0,8,15


In [328]:
df

,Strategy 1,Strategy 2,Score 1,Score 2,Result
0,Student_001,Student_002,62,57,Student_001 wins
1,Student_001,Student_003,64,54,Student_001 wins
2,Student_001,Student_004,60,60,It's a tie
3,Student_001,Student_005,60,60,It's a tie
4,Student_001,Student_006,45,35,Student_001 wins
...,...,...,...,...,...
271,Student_021,Student_023,36,36,It's a tie
272,Student_021,Student_024,66,51,Student_021 wins
273,Student_022,Student_023,24,19,Student_022 wins
274,Student_022,Student_024,32,17,Student_022 wins


In [329]:
# Saving all games' results
df.to_excel(r'Anonymized_Games.xlsx', index=False)
print("Saving was successful!")

Saving was successful!


In [330]:
# Saving the result table
summary_df.to_excel(r'Anonymized_Result_Table.xlsx', index=False)
print("Saving was successful!")

Saving was successful!


In [331]:
print("Number of all played games: ", len(histories))

Number of all played games:  276


An arbitrary history game can be obtained as follows:

In [333]:
Play_number = 7
histories[Play_number]

[('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Defect', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Defect', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate'),
 ('Cooperate', 'Cooperate')]